In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image


from google.colab import drive
import zipfile
import os

# 1️⃣ Mount your Google Drive
drive.mount('/content/drive')

# 2️⃣ Path to your zip file in Drive
zip_path = "/content/drive/MyDrive/archived.zip"
extract_path = "/content/MRI_Images"

# 3️⃣ Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 4️⃣ Check files
os.listdir(extract_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Training', 'Testing']

In [ ]:
# -------------------------
# Config
# -------------------------
data_dir = "/content/MRI_Images"  # Contains Training/ and Testing/
batch_size = 16
img_size = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS = 16
MODEL_DIR = "saved_model"
CLASS_NAMES = ['meningioma', 'glioma  ', 'notumor', 'pituitary']

In [ ]:
# -------------------------
# Dataset loading
# -------------------------
def make_datasets(train_dir, test_dir, img_size=img_size, batch_size=batch_size):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="categorical",  # for multi-class
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir,
        labels="inferred",
        label_mode="categorical",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False
    )

    val_size = int(0.2 * len(train_ds))
    val_ds = train_ds.take(val_size)
    train_ds = train_ds.skip(val_size)

    return (train_ds.prefetch(AUTOTUNE),
            val_ds.prefetch(AUTOTUNE),
            test_ds.prefetch(AUTOTUNE))

In [ ]:
# -------------------------
# Data augmentation
# -------------------------
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
])

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# -------------------------
# Build model
# -------------------------
def build_model(img_size=img_size):
    base_model = MobileNetV2(input_shape=(*img_size, 3),
                             include_top=False,
                             weights='imagenet')
    base_model.trainable = False

    inputs = layers.Input(shape=(*img_size, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(len(CLASS_NAMES), activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model, base_model

# Traing


In [ ]:
# -------------------------
# Training function
# -------------------------
def train_model(train_ds, val_ds):
    model, base_model = build_model()

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    os.makedirs(MODEL_DIR, exist_ok=True)
    checkpoint = ModelCheckpoint(os.path.join(MODEL_DIR, "best_model.h5"),
                                 monitor="val_accuracy", mode="max",
                                 save_best_only=True, verbose=1)
    early = EarlyStopping(monitor="val_accuracy", mode="max", patience=4, restore_best_weights=True, verbose=1)

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint, early]
    )

    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=3,
        callbacks=[checkpoint, early]
    )

    return model


In [ ]:
# -------------------------
# Evaluate model
# -------------------------
def evaluate_model(model, test_ds):
    y_true, y_pred = [], []

    for batch, labels in test_ds:
        preds = model.predict(batch)
        y_pred.extend(np.argmax(preds, axis=1))
        y_true.extend(np.argmax(labels.numpy(), axis=1))

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

In [ ]:
# -------------------------
# Main flow
# -------------------------
if __name__ == "__main__":
    train_dir = os.path.join(data_dir, 'Training')
    test_dir = os.path.join(data_dir, 'Testing')

    train_ds, val_ds, test_ds = make_datasets(train_dir, test_dir)
    model = train_model(train_ds, val_ds)
    evaluate_model(model, test_ds)
    model.save(os.path.join(MODEL_DIR, "final_model.h5"))

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.
Epoch 1/16


KeyboardInterrupt: 

In [ ]:
# -------------------------
# Predict single image
# -------------------------
def predict_image(model, img_path, img_size=(224, 224)):
    try:
        img = image.load_img(img_path, target_size=img_size)
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)[0]  # softmax probabilities
    class_id = np.argmax(preds)
    label = CLASS_NAMES[class_id]
    score = preds[class_id]
    print(f"✅ Prediction complete!\nLabel: {label} ({score*100:.2f}%)")
    return label, preds


# Example prediction
predict_image(model, "/content/MRI_Images/Training/meningioma/Tr-me_0034.jpg")


NameError: name 'model' is not defined